Code based on https://medium.com/@vishnuchirukandathramesh/how-to-run-mistral-7b-on-free-version-of-google-colab-e0effd9c6a12

In [1]:
# Install Dependenies
!pip install -q -U langchain transformers bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 17.3 MB/s eta 0:00:00


In [2]:
# Import Libraries
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Define Quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True)

In [3]:
# Load Model and Tokenizer
model_4bit = AutoModelForCausalLM.from_pretrained( "mistralai/Mistral-7B-Instruct-v0.1", device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [4]:
# Create Hugging Face Pipeline
pipeline_inst = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=5000,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id)

llm = HuggingFacePipeline(pipeline=pipeline_inst)

In [5]:
# Prompt and response function
template = """<s>[INST] Given the text from a financial news article, analyze the content and produce a structured summary categorizing the company's performance and related impacts across multiple predefined categories. Each category should be listed with a corresponding sentiment derived from the article. If a category is not mentioned or relevant based on the article content, mark it as 'N/A'. Ensure that all categories are addressed for a comprehensive summary.

        Categories and Sentiment Options to be Analyzed:
        •	Financial Targets
            •	Greatly Exceeded Target
            •	Exceeded Target
            •	Achieved Target
            •	Below Target
            •	Significantly below Target
        •	Exploration / Discoveries
            •	Major Discovery
            •	Minor Discovery
            •	Favorable Exploration Results
            •	Unfavorable Exploration Results
            •	Exploration Failure
        •	Reserves
            •	Significant Reserves Add
            •	Minor Reserves Add
            •	Stable Reserve Levels
            •	Small Reserves Loss / Writeoff
            •	Significant Reserves Depletions / Writeoff
        •	Production Targets
            •	Greatly Exceeded Target
            •	Exceeded Target
            •	Achieved Target
            •	Below Target
            •	Significantly below Target
        •	New Energy Investments / Projects
            •	Major Advancements in New Energy Initiatives
            •	Minor Advancements in New Energy Initiatives
            •	Setback in New Energies Project
            •	New Energy Projects Abandoned or Failed
        •	Acquisitions and Mergers
            •	Major Acquisition or Merger
            •	Minor Acquisition or Merger
            •	Delay in Acquisition or Merger
            •	Cancelled Acquisition or Merger
        •	Divestments
            •	Major Divestment
            •	Small Divestment
        •	Public Sentiment
            •	Very Positive
            •	Positive
            •	Neutral
            •	Negative
            •	Very Negative
        •	Regulatory / Geopolitical Factors
            •	Favorable change to Operations
            •	Potential Large Disruption to Operations
            •	Potential Small Disruption to Operations
        •	Environmental Factors
            •	Very Positive
            •	Positive
            •	Neutral
            •	Negative
            •	Very Negative

        Output Format:
        •	Use bullet points for each category.
        •	For each category, provide the status or sentiment extracted from the article. If no specific information is available, indicate 'N/A'.

        Example Output:
            •	Financial Targets - Achieved Target
            •	Exploration / Discoveries - N/A
            •	Reserves - Minor Reserves Add
            •	Production Targets - Greatly Exceeded Target
            •	New Energy Investments / Projects - Major Advancements in New Energy Initiatives
            •	Acquisitions and Mergers - Major Acquisition or Merger
            •	Divestments - N/A
            •	Public Sentiment - Very Positive
            •	Regulatory / Geopolitical Factors - Potential Small Disruption to Operations
            •	Environmental Factors - Neutral

        Task:
        •	Parse the article text to identify relevant data points and sentiments for each category.
        •	Summarize the findings in the specified output format.
        •	Maintain accuracy and clarity in categorization and summary presentation.

        **Constraints:** ONLY RESPOND USING THE PROVIDED FORMAT/n/n

        The text from the financial news article is below:
        {article} [/INST] </s>
        """

def generate_response(article):
  prompt = PromptTemplate(template=template, input_variables=["article"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.run({"article":article})
  return response

In [6]:
# Input news article
article = """
Is Chevron Stock A Buy Or A Sell After Revealing $53 Billion Hess Deal Is At Risk?
FacebookTwitterLinkedInShare Licensing
KIT NORTON10:48 AM ET 02/28/2024
Dow Jones energy giant Chevron (CVX) was a strong candidate for investors looking to grab a piece of the 2022 energy bonanza, with CVX rising more than 50% even as the broader stock market declined. However, Chevron stock dropped 17% in 2023, and is now meeting resistance at its 200-day moving average as sagging oil and natural gas prices have whittled away at energy company profits and revenue. Is CVX a buy or a sell?

X


Meanwhile, on Monday the energy giant warned that its blockbuster $53 billion deal to buy Hess (HES) could be upended by Exxon Mobil (XOM) and China's Cnooc. Both companies are asserting they have a right to counter Chevron's bid for Hess's stake in Guyana.

Hess and Chevron have been engaged in discussions with Exxon Mobil and China National Offshore Oil Corporation (Cnooc) regarding a right of first refusal provision in the joint operating agreement for the Stabroek Block offshore Guyana, according to a Feb. 26 Chevron regulatory filing.

"If these discussions do not result in an acceptable resolution and arbitration (if pursued) does not result in a confirmation that such right of first refusal provision is inapplicable to the merger, then there would be a failure of a closing condition under the merger agreement, in which case the merger would not close," Chevron wrote.


Chevron stock edged down 2% while Hess shares dropped 3% Tuesday during market action.

CVX Entering 2024
Chevron stock started 2024 by falling more than 1% in January. However, shares have angled slightly higher in February. Meanwhile, both Chevron and Exxon Mobil closed the door on 2023 in early February with mixed earnings and revenue reports. For the 2024 year, both companies forecast nearly flat oil production compared to 2023 levels.

The two supermajors also enter 2024 after making mega deals in the latter part of 2023, which they got done without taking on any new debt.

Exxon Mobil significantly expanded its Permian Basin holdings when it paid $60 billion for Pioneer Natural Resources (PXD) in October, 2023. Chevron quickly followed up with its $53 billion purchase of Hess, bolstering its share of an Exxon-led project offshore from Guyana.

What Do The Exxon Mobil And Chevron Deals Say About The Oil Industry's Future?

Buffett And Chevron
Warren Buffett and Berkshire Hathaway (BRKB) reduced its CVX holdings in 2023. Berkshire Hathaway currently has a 6.7% stake (more than 126 million shares) in Chevron. Warren Buffett began buying the Dow Jones stock in 2020. Prior to Buffett's decision to sell CVX, beginning in Q1 2023, Berkshire Hathaway amassed a stake totaling 167 million CVX shares, worth around $29 billion.

In 2022, as the U.S. economy bounced from Covid pandemic retreat to inflation-driven growth, Russia invaded Ukraine in February. The combination sent oil, gasoline and natural gas prices soaring. This resulted in Chevron, Exxon Mobil and other energy stocks leading the market, with the companies racking up record profits.

However, Chevron investors had a tougher year in 2023 as oil markets turned lower after the Russia/Ukraine price spikes of 2022.

Chevron To Report Fourth-Quarter Earnings
The Dow Jones stock announced fourth-quarter earnings and revenue on Feb. 2. Chevron reported Q4 EPS of $3.45 with revenue totaling $47.18 billion. Analysts had predicted earnings of $3.19 per share and sales of $50.92 billion. For the full year, Chevron earnings fell 30% to $13.13 per share with revenue declining 15% to $200.95 billion. Chevron also increased its quarterly dividend by 8% to $1.63, from $1.51.

Wall Street also expects EPS to fall again in 2024 to $12.75 with sales hitting $198.51 billion.

Chevron Stock: 2024 Outlook
Chevron plans to raise its capital spending in 2024. It targets global expenditures in a range of $18.5 billion to $19.5 billion, up 11% from $17 billion in 2023, according to its Dec. 6 corporate plan. This increase in capital spending comes as CVX shelled out $53 billion for Hess and purchased PDC Energy for $6.3 billion in 2023.

After the Hess acquisition, Chevron also predicts capital spending will run at between $19 billion and $22 billion a year through 2027. That's up from its pre-Hess forecast of $14 billion -$16 billion. Wall Street forecasts CVX's 2024 earnings totaling $14.46 per share, down 23% compared to 2022, with sales dropping 15% to $201 billion.

What Does 2024 Hold For The Energy Industry and The Permian Basin?

Chevron expects 2024 upstream spending to be about $14 billion, with two-thirds of that allocated to the U.S., including around $5 billion for Chevron's Permian Basin development.

CVX forecasts downstream capital expenditures to be roughly $1.5 billion, with 80% bookmarked for the U.S. Meanwhile, Chevron added it has about $2 billion in spending for "lower carbon" projects, in line with 2023 levels.

"We're maintaining capital discipline in both traditional and new energies," Chevron Chief Executive Mike Wirth said in the Dec. 6 announcement.

Looking to 2024, Chevron announced in early February it expects production to increase 4%-7% compared to its 3.12 million barrels of oil equivalent per day in 2023. Analysts had been projecting a 14% production increase for Chevron.

The U.S. supermajor, at its annual investor day in February 2023, ahead of its acquisition moves, outlined plans to have total oil and gas production increase around 3% above current levels by 2027.

Chevron Revenue Slips
The Dow Jones stock reported third-quarter earnings falling 45% to $3.05 per share with revenue declining 19% to $54.08 billion. The company said the EPS decrease was due to lower upstream realizations and lower margins on refined product sales. Meanwhile, Chevron added its revenue drop came on the back of lower oil and gas prices.

Chevron's worldwide net oil-equivalent production was up 4% compared to 2022 mostly because of its PDC Energy deal. Oil production in the U.S. totaled 1.41 million barrels of oil equivalent per day, up 20% vs. 2022, due to the addition of PDC's 179,000 oil-equivalent barrels per day and production increases in the Permian Basin.

Chevron's refinery product sales also jumped 4% on higher demand for jet fuel in the quarter.

In the second quarter, Chevron earnings dropped nearly 50% to $3.08 per share with revenue of $48.89 billion, down 29% vs. Q2 2022. In Q1, Chevron topped Wall Street estimates, even as revenue slipped compared to a year ago. Earnings gained 6% to $3.55 per share in the first quarter. Sales dropped 6% to $50.79 billion. Higher margins on refined product sales drove the earning beat, the company said, partially offset by lower oil prices and rising production costs.

CVX: Oil Price Uncertainty
U.S. oil futures have been hovering above $77 per barrel in recent days. Goldman Sachs forecasts Brent crude prices between $70-$90 per barrel in 2024, cutting its expectation by $10 per barrel. The price of Brent crude, widely used as the global benchmark and which originates in Europe's North Sea fields, typically runs several dollars higher per barrel than West Texas Intermediate.

"While we have adjusted the range, we still look for rangebound prices and only moderate price volatility in 2024," the firm wrote. "Elevated spare capacity to handle tightening shocks should limit upside price moves."

What's In Store For Exxon Mobil And Chevron In 2024?

Meanwhile, Third Bridge analyst Peter McNally has said he expects an uncertain year, with Brent crude trading in the range of $70-$95 per barrel in 2024.

That status of China's economy, and its considerable demand for oil, also remains a large question mark for analysts heading into 2024.

CFRA's Stewart Glickman sees current oil prices lower than they should be considering the current political picture, including China's economic outlook and the Organization of Petroleum Exporting Countries (OPEC) latest production quota action. Those factors should lift oil prices to a 2024 average somewhere closer to $90 per barrel, Glickman says.

"I am still reasonably bullish on crude," Glickman said. "Not necessarily a great year, but a good year."

Is Chevron Stock A Buy Right Now?
Chevron stock dropped below its 200-day and 50-day averages in October 2023 when shares sank nearly 23%. The Dow Jones stock edged lower in November, dropping another 2%. Shares angled slightly higher in December but came back down in January.


As of Feb. 28, CVX has gained 4% on the month and has met resistance at its 200-day moving line.

Chevron stock's relative strength line, which gauges its strength against the overall S&P 500, has been on a downtrend and is currently 39.

Chevron stock is not a buy right now.
"""

In [7]:
# Test Function
full_response = generate_response(article)
split_response = full_response.split("</s>", 1)
final_response = split_response[1]
print(final_response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 
        
Summary:
        
        Financial Targets
            •	Achieved Target
            •	Below Target
            •	Greatly Exceeded Target
            N/A: Significantly below Target
            N/A: Small Reserves Loss / Writeoff

        Exploration / Discoveries
            •	Major Discovery
            •	Minor Discovery
            •	Favorable Exploration Results
            •	Unfavorable Exploration Results

        Reserves
            •	Significant Reserves Add
            •	Minor Reserves Add
            •	Stable Reserve Levels
            •	Small Reserves Loss / Writeoff
            N/A: Significant Reserves Depletions / Writeoff

        Production Targets
            •	Greatly Exceeded Target
            •	Exceeded Target
            •	Achieved Target
            •	Below Target
            •	Significantly below Target

        New Energy Investments / Projects
            •	Major Advancements in New Energy Initiatives
            •	Minor Advancements in New Energy